In [1]:
!pip install tensorflow_hub tensorflow_text pytorch_lightning evaluate datasets -q


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
error: uninstall-distutils-installed-package

× Cannot uninstall TBB 0.2
╰─> It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [6]:
!pip install pytorch-lightning

  Using cached pytorch_lightning-2.5.0.post0-py3-none-any.whl.metadata (21 kB)
  Using cached torchmetrics-1.6.1-py3-none-any.whl.metadata (21 kB)
  Using cached lightning_utilities-0.12.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached aiohttp-3.11.11-cp39-cp39-win_amd64.whl.metadata (8.0 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached frozenlist-1.5.0-cp39-cp39-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.1.0-cp39-cp39-win_amd64.whl.metadata (5.1 kB)
  Using cached propcache-0.2.1-cp39-cp39-win_amd64.whl.metadata (9.5 kB)
  Using cached yarl-1.18.3-cp39-cp39-win_amd64.whl.metadata (71 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.13.1-py3-none-win_amd64.


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
error: uninstall-distutils-installed-package

× Cannot uninstall TBB 0.2
╰─> It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [7]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

from torch.utils.data import DataLoader
from datasets import Dataset, load_dataset
import numpy as np
import evaluate

from typing import List, Dict

ModuleNotFoundError: No module named 'pytorch_lightning'

In [12]:
class Model(pl.LightningModule):
    """
    A PyTorch Lightning module that encapsulates a multilayer perceptron model.

    Args:
        hidden_dims (List[int], optional): A list of integers representing the number of hidden units
            in each layer. Defaults to [768, 128].
        dropout_prob (float, optional): The dropout probability for regularization. Defaults to 0.5.
        learning_rate (float, optional): The learning rate for the optimizer. Defaults to 1e-3.
    """
    def __init__(self,
                 hidden_dims: List[int] = [768, 128],
                 dropout_prob: float = 0.5,
                 learning_rate: float = 1e-3):
        super().__init__()
        self.train_acc = evaluate.load("accuracy")
        self.val_acc = evaluate.load("accuracy")
        self.test_acc = evaluate.load("accuracy")
        self.hidden_dims = hidden_dims
        self.dropout_prob = dropout_prob
        self.learning_rate = learning_rate

        self.embedding_dim = 512

        layers = []
        prev_dim = self.embedding_dim

        if dropout_prob > 0:
            layers.append(nn.Dropout(dropout_prob))

        for h in hidden_dims:
            layers.append(nn.Linear(prev_dim, h))
            prev_dim = h
            if dropout_prob > 0:
                layers.append(nn.Dropout(dropout_prob))
            layers.append(nn.ReLU())
            if dropout_prob > 0:
                layers.append(nn.Dropout(dropout_prob))
        # output layer
        layers.append(nn.Linear(prev_dim, 2))

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        """
        Forward pass through the model.

        Args:
            x (torch.Tensor): A batch of input embeddings.

        Returns:
            torch.Tensor: Output logits.
        """
        # x will be a batch of USEm vectors
        logits = self.layers(x)
        return logits

    def configure_optimizers(self):
        """
        Configure the Adam optimizer for the model.

        Returns:
            torch.optim.Optimizer: The configured optimizer.
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)  # type: ignore
        return optimizer

    def __compute_loss(self, batch):
        """
        Compute the cross-entropy loss and predictions for a batch.

        Args:
            batch (Dict[str, torch.Tensor]): A dictionary containing a batch of data.
                Expected keys are "embedding" and "label".

        Returns:
            Tuple[torch.Tensor, torch.Tensor, torch.Tensor]: The computed loss,
                predictions and true labels.
        """
        x, y = batch["embedding"], batch["label"]
        logits = self(x)
        preds = torch.argmax(logits, dim=1).detach().cpu().numpy()
        loss = F.cross_entropy(logits, y)
        return loss, preds, y

    def training_step(self, batch, batch_idx):
        """
        Performs a single training step, calculating the loss and other metrics.

        Args:
            batch (Dict[str, torch.Tensor]): The input data for this batch, which includes
            the sentences and their labels.
            batch_idx (int): The index of this batch.

        Returns:
            torch.Tensor: The loss for this batch.
        """
        loss, preds, y = self.__compute_loss(batch)
        self.train_acc.add_batch(predictions=preds, references=y)
        acc = self.train_acc.compute()["accuracy"] # type: ignore
        values = {"train_loss": loss, "train_accuracy": acc}
        self.log_dict(values, on_step=True, on_epoch=True,
                      prog_bar=True, logger=True)
        return loss
    ''' on_epoch argument in log_dict automatically accumulates and logs at the end of the epoch.  '''

    def validation_step(self, batch, batch_idx):
        """
        Performs a single validation step, calculating the loss and other metrics.

        Args:
            batch (Dict[str, torch.Tensor]): The input data for this batch, which includes
            the sentences and their labels.
            batch_idx (int): The index of this batch.

        Returns:
            torch.Tensor: The loss for this batch.
        """
        loss, preds, y = self.__compute_loss(batch)
        self.val_acc.add_batch(predictions=preds, references=y)
        acc = self.val_acc.compute()["accuracy"]    # type: ignore
        values = {"val_loss": loss, "val_accuracy": acc}
        self.log_dict(values, on_step=True, on_epoch=True,
                      prog_bar=True, logger=True)
        return loss


    def test_step(self, batch, batch_idx):
        """
        Performs a single test step, calculating the loss and other metrics.

        Args:
            batch (Dict[str, torch.Tensor]): The input data for this batch, which includes
            the sentences and their labels.
            batch_idx (int): The index of this batch.

        Returns:
            torch.Tensor: The loss for this batch.
        """
        loss, preds, y = self.__compute_loss(batch)
        self.test_acc.add_batch(predictions=preds, references=y)
        acc = self.test_acc.compute()["accuracy"]   # type: ignore
        values = {"test_loss": loss, "test_accuracy": acc}
        self.log_dict(values, on_step=False, on_epoch=True,
                      prog_bar=True, logger=True)
        return loss

In [13]:
model_URL = 'https://tfhub.dev/google/universal-sentence-encoder-large/5'

encoder = hub.load(model_URL)

def embed_text(text: List[str]) -> List[np.ndarray]:
    """
    Used to generate vector representations of text using an encoder
    Embeds a list of text strings using a pre-defined 'encoder' and converts them into numpy arrays.

    Args:
        text (List[str]): List of text strings to be embedded.

    Returns:
        List[np.ndarray]: List of numpy arrays representing the embedded texts.
    """
    vectors = encoder(text)
    return [vector.numpy() for vector in vectors]
''' Want the vectors to be numpy arrays, not Tensorflow tensors, b/c they'll be used in PyTorch. '''

def encoder_factory(label2int: Dict[str, int]):
    """
    Creates a custom function to process a batch of data. The returned function, encode, transforms a batch by embedding its text and encoding its labels according to the provided mapping label2int.

    Args:
        label2int (Dict[str, int]): A dictionary that maps class labels to integers.

    Returns:
        Function: A function that takes a batch of data and returns the batch with
                  text data embedded and labels encoded to integers.
    """
    def encode(batch):
        """
        Encode a batch of data.

        Args:
            batch (Dict[str, Union[List[str], List[int]]]): A dictionary containing a batch of data.
                Expected keys are "text" and "label".

        Returns:
            Dict[str, Union[List[np.ndarray], List[int]]]: The input batch with text data embedded and labels encoded.
        """
        batch["embedding"] = embed_text(batch["text"])
        batch["label"] = [label2int[str(x)] for x in batch["label"]]
        return batch

    return encode

class YelpDataLoader(pl.LightningDataModule):
    """
    YelpDataLoader is a PyTorch Lightning DataModule for the Yelp Polarity Dataset.
    It loads the dataset, applies preprocessing steps, and creates DataLoaders for training, validation, and test sets.
    """
    def __init__(self,
                 batch_size: int = 32,
                 num_workers: int = 2):
        """
        Args:
            batch_size (int, optional): Size of the mini-batches. Defaults to 32.
            num_workers (int, optional): How many subprocesses to use for data loading. 0 means that the data will be loaded in the main process. Defaults to 2.
        """
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.pin_memory = torch.cuda.is_available()  # type: ignore

    def prepare_data(self):
        ''' This method loads a subset of the train and test sets.
        It uses the first 2% of the train and test sets to
        train and test, respectively.
        It uses the last 1% of the training set for validation.
        '''
        self.test_ds = load_dataset('yelp_polarity', split="test[:2%]")
        self.train_ds = load_dataset('yelp_polarity', split="train[:2%]")
        self.val_ds = load_dataset('yelp_polarity', split="train[99%:]")

        self.label_names = self.train_ds.unique("label")
        label2int = {str(label): n for n, label in enumerate(self.label_names)}
        self.encoder = encoder_factory(label2int)

    def setup(self):
        """
        This method is called on every GPU when using DDP (Distributed Data Parallelism) and it's the right place to define
        transformations and dataset splits.
        Here, it applies the encoder to the train, validation and test datasets and sets the dataset format to PyTorch.
        """
        # Compute embeddings in batches, so that they fit in the GPU's RAM.
        self.train = self.train_ds.map(self.encoder, batched=True, batch_size=self.batch_size)
        self.train.set_format(type="torch", columns=["embedding", "label"],                     # type: ignore
                              output_all_columns=True)

        self.val = self.val_ds.map(self.encoder, batched=True, batch_size=self.batch_size)
        self.val.set_format(type="torch", columns=["embedding", "label"],                       # type: ignore
                            output_all_columns=True)

        self.test = self.test_ds.map(self.encoder, batched=True, batch_size=self.batch_size)
        self.test.set_format(type="torch", columns=["embedding", "label"],                      # type: ignore
                             output_all_columns=True)

    def train_dataloader(self):
        """
        DataLoader for the training set.
        Returns:
            DataLoader: DataLoader for the training set.
        """
        return DataLoader(self.train,                                                           # type: ignore
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          pin_memory=self.pin_memory,
                          shuffle=True)

    def val_dataloader(self):
        """
        DataLoader for the validation set.
        Returns:
            DataLoader: DataLoader for the validation set.
        """
        return DataLoader(self.val,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          pin_memory=self.pin_memory)

    def test_dataloader(self):
        """
        DataLoader for the test set.
        Returns:
            DataLoader: DataLoader for the test set.
        """
        return DataLoader(self.test,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers)

    def setup_metrics(self):
        """
        Setup for metrics, using the 'evaluate' library.
        """
        self.train_acc = evaluate.load("accuracy")


In [14]:
pl.seed_everything(445326, workers=True)

INFO:lightning_fabric.utilities.seed:Seed set to 445326


445326

In [15]:
model_URL = 'https://tfhub.dev/google/universal-sentence-encoder-large/5'

encoder = hub.load(model_URL)

In [16]:
data = YelpDataLoader()
data.prepare_data()

In [17]:
data.setup()
print(len(data.train))
print(len(data.val))
print(len(data.test))

Map:   0%|          | 0/11200 [00:00<?, ? examples/s]

Map:   0%|          | 0/5600 [00:00<?, ? examples/s]

Map:   0%|          | 0/760 [00:00<?, ? examples/s]

11200
5600
760


In [18]:
model = Model()

In [19]:
MAX_EPOCHS = 50

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor="val_loss",
    dirpath="model",
    filename="yelp-sentiment-multilingual-{epoch:02d}-{val_loss:.3f}",
    save_top_k=3,
    mode="min"
)

# Update trainer for newer PyTorch Lightning versions
trainer = pl.Trainer(
    accelerator="gpu",  # Use GPU
    devices=1,  # Number of GPUs to use
    max_epochs=MAX_EPOCHS,
    callbacks=[checkpoint_callback]
)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [20]:
trainer.fit(model, data.train_dataloader(), data.val_dataloader())

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name   | Type       | Params | Mode 
----------------------------------------------
0 | layers | Sequential | 492 K  | train
----------------------------------------------
492 K     Trainable params
0         Non-trainable params
492 K     Total params
1.971     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training: |          | 0/? [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validation: |          | 0/? [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=25` reached.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [21]:
trainer.test(dataloaders=data.test_dataloader())

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/model/yelp-sentiment-multilingual-epoch=24-val_loss=0.190.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/model/yelp-sentiment-multilingual-epoch=24-val_loss=0.190.ckpt
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid

Testing: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 24. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8986842036247253     │
│         test_loss         │    0.24877344071865082    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.24877344071865082, 'test_accuracy': 0.8986842036247253}]

In [22]:
best_model = Model.load_from_checkpoint(checkpoint_callback.best_model_path)

In [ ]:
trainer.save_checkpoint("model/yelp_sentiment_multilingual.ckpt")
best_model = Model.load_from_checkpoint("model/yelp_sentiment_multilingual.ckpt")

In [23]:
from pprint import PrettyPrinter
pp = PrettyPrinter()

In [24]:
import pprint as pp

def predict_sentiments(text_list):
    """
    Predict the sentiment of the input texts and return results in the desired format.

    Args:
        text_list (List[str]): List of text samples to classify.

    Returns:
        List[Dict]: List of dictionaries with keys 'label', 'score', and 'text'.
    """
    # Get the embeddings for the input texts
    embeddings = torch.tensor(embed_text(text_list)).to(model.device)  # Embed the texts and move to model's device

    # Forward pass through the model to get logits
    with torch.no_grad():
        logits = model(embeddings)

    # Convert logits to probabilities using softmax
    probs = F.softmax(logits, dim=1).cpu().numpy()

    # Convert probabilities to labels (0: negative, 1: positive)
    labels = ["negative" if prob[0] > prob[1] else "positive" for prob in probs]

    # Prepare the output in the desired format
    output = []
    for i, text in enumerate(text_list):
        output.append({
            'label': labels[i],
            'score': float(np.max(probs[i])),  # Highest probability score
            'text': text
        })

    return output

In [25]:
english_text = "Like any Barnes & Noble, it has a nice comfy cafe, and a large selection of books. The staff is very friendly and helpful. They stock a decent selection, and the prices are pretty reasonable."

german_translation = "Wie jedes Barnes & Noble hat es ein nettes, gemütliches Café und eine große Auswahl an Büchern. Das Personal ist sehr freundlich und hilfsbereit. Sie haben eine anständige Auswahl und die Preise sind ziemlich vernünftig."

# Get predictions
predictions = predict_sentiments([english_text, german_translation])

# Pretty print the predictions
pp.pprint(predictions)

[{'label': 'positive',
  'score': 0.9977879524230957,
  'text': 'Like any Barnes & Noble, it has a nice comfy cafe, and a large '
          'selection of books. The staff is very friendly and helpful. They '
          'stock a decent selection, and the prices are pretty reasonable.'},
 {'label': 'positive',
  'score': 0.876610279083252,
  'text': 'Wie jedes Barnes & Noble hat es ein nettes, gemütliches Café und '
          'eine große Auswahl an Büchern. Das Personal ist sehr freundlich und '
          'hilfsbereit. Sie haben eine anständige Auswahl und die Preise sind '
          'ziemlich vernünftig.'}]


<ipython-input-24-d33758c1487d>:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  embeddings = torch.tensor(embed_text(text_list)).to(model.device)  # Embed the texts and move to model's device


In [38]:
english_text = "Nice cafe restaurant in Hilton Colombo. Usually has good breakfast and lunch buffet spreads. Staff is nice and courteous and always smiling. Menu can improve a bit"

italian_translation = "Bel ristorante-caffetteria all'Hilton Colombo. Di solito offre buoni buffet per colazione e pranzo. Il personale è gentile, cortese e sempre sorridente. Il menù potrebbe migliorare un po'"

sinhala_translation = "koḷam̆ba hilṭan hi lassana kæfē avanhalak. sāmānyayen hon̆da udǣsana āhāraya saha divā āhāraya bufē pætira æta. kārya maṇḍalaya hon̆da saha ācāraśīlī vana atara sǣma viṭama sināseyi. menuva ṭikak væḍi diyuṇu kaḷa hækiya"

# Get predictions
predictions = predict_sentiments([english_text, german_translation,sinhala_translation])

# Pretty print the predictions
pp.pprint(predictions)

[{'label': 'positive',
  'score': 0.9994372725486755,
  'text': 'Nice cafe restaurant in Hilton Colombo. Usually has good breakfast '
          'and lunch buffet spreads. Staff is nice and courteous and always '
          'smiling. Menu can improve a bit'},
 {'label': 'positive',
  'score': 0.9341564178466797,
  'text': 'Wie jedes Barnes & Noble hat es ein nettes, gemütliches Café und '
          'eine große Auswahl an Büchern. Das Personal ist sehr freundlich und '
          'hilfsbereit. Sie haben eine anständige Auswahl und die Preise sind '
          'ziemlich vernünftig.'},
 {'label': 'positive',
  'score': 0.8991713523864746,
  'text': 'koḷam̆ba hilṭan hi lassana kæfē avanhalak. sāmānyayen hon̆da '
          'udǣsana āhāraya saha divā āhāraya bufē pætira æta. kārya maṇḍalaya '
          'hon̆da saha ācāraśīlī vana atara sǣma viṭama sināseyi. menuva ṭikak '
          'væḍi diyuṇu kaḷa hækiya'}]
